In [0]:
 import datetime as dt
 from zoneinfo import ZoneInfo

 from alpaca_pyspark import HistoricalBarsDataSource

 spark.dataSource.register(HistoricalBarsDataSource)

In [0]:
tz = ZoneInfo("America/New_York")

hist_bars_opts = {
    "symbols": ["AAPL", "MSFT", "GOOG"],
    'APCA-API-KEY-ID': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_API_KEY"),
    'APCA-API-SECRET-KEY': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_SECRET_KEY"),
    "timeframe": "1Day",
    "start": dt.datetime(2021, 1, 1, tzinfo=tz).isoformat(),
    "end": dt.datetime(2022, 1, 1, tzinfo=tz).isoformat(),
    "limit": 1000
}

In [0]:
hist_bars_df = (spark.read
                .format("Alpaca_HistoricalBars")
                .options(**hist_bars_opts)
                .load())

In [0]:
display(hist_bars_df)

# DEBUG

In [0]:
from alpaca_pyspark.bars import HistoricalBarsReader

bars_schema = """
    symbol: STRING,
    time: TIMESTAMP,
    open: FLOAT,
    high: FLOAT,
    low: FLOAT,
    close: FLOAT,
    volume: INT,
    trade_count: INT,
    vwap: FLOAT
"""

hist_bars_reader = HistoricalBarsReader(bars_schema, hist_bars_opts)

In [0]:
dt.datetime(2021, 1, 1, tzinfo=tz).isoformat()

In [0]:
sym_parts = hist_bars_reader.partitions()

In [0]:
sym_parts[0].symbol

In [0]:
bars_iter = hist_bars_reader.read(sym_parts[0])

In [0]:
for bar in bars_iter:
    print(bar)

In [0]:
import requests

headers = {
    'Content-Type': 'application/json',
    'APCA-API-KEY-ID': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_API_KEY"),
    'APCA-API-SECRET-KEY': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_SECRET_KEY")
}

test_uri = "https://data.alpaca.markets/v2/stocks/bars?timeframe=1Day&start=2021-01-01T00%3A00%3A00&end=2022-01-01T00%3A00%3A00&limit=1000&symbols=AAPL"

In [0]:
test_uri

In [0]:
resp = requests.get(test_uri, headers=headers)
resp.status_code

In [0]:
resp.raise_for_status()

In [0]:
resp.text